In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import seaborn as sns; sns.set()  # for plot styling

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.preprocessing import MinMaxScaler, minmax_scale

from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D # for 3D plots

from sklearn.cluster import MeanShift, estimate_bandwidth

from itertools import cycle
from sklearn.mixture import GaussianMixture

import statsmodels.api as sm

import statsmodels.formula.api as smf

# Composicion de pipelines
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PowerTransformer
# Regresion lineal
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression

# Importar/ Exportar modelos
from joblib import dump, load

# Metricas
from sklearn.metrics import mean_squared_error as mse

# q-q plots
import scipy.stats as stats

import statsmodels.api as sm

import statsmodels.formula.api as smf

from pandas_profiling import ProfileReport

# Para preparar los datos
from sklearn.preprocessing import LabelEncoder

# Para crear el arbol de decisión 
from sklearn.tree import DecisionTreeClassifier 

# Para usar KNN como clasificador
from sklearn.neighbors import KNeighborsClassifier

# Para usar Random Tree Forest como clasificador
from sklearn.ensemble import RandomForestClassifier

# Para realizar la separación del conjunto de aprendizaje en entrenamiento y test.
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn import model_selection

# Para evaluar el modelo
from sklearn.metrics import confusion_matrix, classification_report, precision_score, recall_score, f1_score, accuracy_score
from sklearn.metrics import plot_confusion_matrix

# Para búsqueda de hiperparámetros
from sklearn.model_selection import GridSearchCV

# Para la validación cruzada
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold

#Librerías para la visualización
import matplotlib.pyplot as plt

# Seaborn
import seaborn as sns 
from sklearn import tree
from sklearn.preprocessing import LabelEncoder

#SMF
import statsmodels.formula.api as smf

from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVR

# check version number
import imblearn

from collections import Counter

from imblearn.over_sampling import SMOTE, ADASYN
from sklearn import linear_model

In [2]:
datos_inglaterra= pd.read_excel("./data/pruebas/Info_premier_compl_2022-2023.xlsx",sheet_name=None)
datos_espana= pd.read_excel("./data/pruebas/Info_laliga_compl_2022-2023.xlsx",sheet_name=None)
datos_italia= pd.read_excel("./data/pruebas/Info_serie_a_italia_compl_2022-2023.xlsx",sheet_name=None)
datos_francia= pd.read_excel("./data/pruebas/Info_ligue1_compl_2022-2023.xlsx",sheet_name=None)
datos_alemania= pd.read_excel("./data/pruebas/Info_bundesliga_compl_2022-2023.xlsx",sheet_name=None)
datos_brasil= pd.read_excel("./data/pruebas/Info_seriea_brasil_compl_2022-2023.xlsx",sheet_name=None)

In [3]:
datos_brasil= pd.read_excel("./data/pruebas/Info_seriea_brasil_compl_2022-2023.xlsx",sheet_name=None)

In [4]:
datos_brasil_shots= datos_brasil['stats_shots'].copy()
datos_brasil_matchs= datos_brasil['match_stats'].copy()

In [5]:
def limpiar_fecha(entrada):
    if "\nClássico" in entrada or "\nClassico" in entrada:
        return entrada.split("\n")[0].strip()
    return entrada.strip()

In [6]:
def limpiar_espacios(entrada):
    return entrada.strip()

In [7]:
datos_brasil_shots=datos_brasil_shots[~datos_brasil_shots['Equipo'].isnull()]

In [8]:
datos_brasil_matchs['Fecha'] = datos_brasil_matchs['Fecha'].apply(limpiar_fecha)
datos_brasil_matchs['HomeTeam'] = datos_brasil_matchs['HomeTeam'].str.strip()
datos_brasil_matchs['AwayTeam'] = datos_brasil_matchs['AwayTeam'].str.strip()
datos_brasil_shots['Fecha'] = datos_brasil_shots['Fecha'].apply(limpiar_fecha)
datos_brasil_shots['Equipo'] = datos_brasil_shots['Equipo'].str.strip()
# datos_brasil_shots['Area_Ono']=np.where(datos_brasil_shots['Distance']<=17,"En el area","Fuera del area")
# datos_brasil_shots['EnelArea']=np.where(datos_brasil_shots['Distance']<=17,1,0)
# datos_brasil_shots['NoEnelArea']=np.where(datos_brasil_shots['Distance']>17,1,0)

In [9]:
datos_brasil_shots.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5091 entries, 0 to 5277
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        5091 non-null   int64  
 1   Fecha             5091 non-null   object 
 2   Minute            5091 non-null   object 
 3   Jugador           5091 non-null   object 
 4   Equipo            5091 non-null   object 
 5   xG                5091 non-null   float64
 6   PSxG              1707 non-null   float64
 7   Resultado         5091 non-null   object 
 8   Distance          5091 non-null   float64
 9   Parte del cuerpo  5091 non-null   object 
 10  Notas             860 non-null    object 
 11  Jugador.1         4802 non-null   object 
 12  Evento            4836 non-null   object 
 13  Jugador.2         4041 non-null   object 
 14  Evento.1          4054 non-null   object 
dtypes: float64(3), int64(1), object(11)
memory usage: 636.4+ KB


In [10]:
datos_brasil_shots= datos_brasil_shots.merge(datos_brasil_matchs[['Fecha','HomeTeam']],right_on=['Fecha','HomeTeam'],left_on=['Fecha','Equipo'],how="left")
datos_brasil_shots.info()
# prueba.merge(datos_brasil_matchs[['Fecha','AwayTeam']],right_on=['Fecha','AwayTeam'],left_on=['Fecha','Equipo.1'],how="left").info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5130 entries, 0 to 5129
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        5130 non-null   int64  
 1   Fecha             5130 non-null   object 
 2   Minute            5130 non-null   object 
 3   Jugador           5130 non-null   object 
 4   Equipo            5130 non-null   object 
 5   xG                5130 non-null   float64
 6   PSxG              1717 non-null   float64
 7   Resultado         5130 non-null   object 
 8   Distance          5130 non-null   float64
 9   Parte del cuerpo  5130 non-null   object 
 10  Notas             863 non-null    object 
 11  Jugador.1         4837 non-null   object 
 12  Evento            4873 non-null   object 
 13  Jugador.2         4069 non-null   object 
 14  Evento.1          4084 non-null   object 
 15  HomeTeam          2956 non-null   object 
dtypes: float64(3), int64(1), object(12)
memory

In [11]:
datos_brasil_shots['LocalOVisita']= np.where(datos_brasil_shots['HomeTeam'].isnull(),"Visitante","Local")

In [12]:
datos_brasil_shots['xGReal'] = np.where(~datos_brasil_shots['PSxG'].isnull(),datos_brasil_shots['PSxG'],datos_brasil_shots['xG'])
datos_brasil_shots['Goles'] = np.where(datos_brasil_shots['Resultado']=="Gol",1,0)

In [13]:
datos_brasil_shots

,Unnamed: 0,Fecha,Minute,Jugador,Equipo,xG,PSxG,Resultado,Distance,Parte del cuerpo,Notas,Jugador.1,Evento,Jugador.2,Evento.1,HomeTeam,LocalOVisita,xGReal,Goles
0,0,"Sábado Abril 15, 2023",2,Gabriel Pirani,Fluminense,0.28,0.27,Salvado,9.0,Pie derecho,NaN,John Kennedy,Pase (vivo),John Kennedy,Toma,NaN,Visitante,0.27,0
1,1,"Sábado Abril 15, 2023",4,Juninho,América (MG),0.25,0.69,Salvado,9.0,Pie izquierdo,NaN,Henrique Almeida,Pase (vivo),NaN,NaN,América (MG),Local,0.69,0
2,1,"Sábado Abril 15, 2023",4,Juninho,América (MG),0.25,0.69,Salvado,9.0,Pie izquierdo,NaN,Henrique Almeida,Pase (vivo),NaN,NaN,América (MG),Local,0.69,0
3,2,"Sábado Abril 15, 2023",4,Iago Maidana,América (MG),0.06,NaN,Bloqueado,17.0,Pie derecho,Voleibol,Juninho,Tiro,NaN,NaN,América (MG),Local,0.06,0
4,2,"Sábado Abril 15, 2023",4,Iago Maidana,América (MG),0.06,NaN,Bloqueado,17.0,Pie derecho,Voleibol,Juninho,Tiro,NaN,NaN,América (MG),Local,0.06,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5125,284,"Lunes Agosto 14, 2023",90+1,Jadsom,Bragantino,0.01,0.10,Salvado,25.0,Pie derecho,NaN,Bruno,Toma,Lucas Evangelista,Pase (vivo),Bragantino,Local,0.10,0
5126,285,"Lunes Agosto 14, 2023",90+2,Juninho Capixaba,Bragantino,0.03,NaN,Salvado del ataque,27.0,Pie izquierdo,Voleibol,Matheus Gonçalves,Pase (vivo),Jadsom,Tiro,Bragantino,Local,0.03,0
5127,286,"Lunes Agosto 14, 2023",90+4,Alerrandro,Bragantino,0.08,NaN,Bloqueado,18.0,Pie derecho,NaN,Jadsom,Pase (vivo),NaN,NaN,Bragantino,Local,0.08,0
5128,287,"Lunes Agosto 14, 2023",90+5,Bruno,Bragantino,0.03,NaN,Bloqueado,15.0,Pie derecho,NaN,Marcos Vinicios Sorriso,Pase (vivo),Bruno,Pase (vivo),Bragantino,Local,0.03,0


In [14]:
tiros_reformados_brasil=datos_brasil_shots.groupby(by=['Fecha','Equipo']).sum().reset_index()[['Fecha','Equipo','xG','PSxG','xGReal','Goles']]
tiros_reformados_brasil

,Fecha,Equipo,xG,PSxG,xGReal,Goles
0,"Domingo Abril 16, 2023",Corinthians,1.55,2.25,2.42,2
1,"Domingo Abril 16, 2023",Coritiba,0.37,0.42,0.61,0
2,"Domingo Abril 16, 2023",Cruzeiro,0.55,0.54,0.81,1
3,"Domingo Abril 16, 2023",Flamengo,1.38,2.60,2.87,3
4,"Domingo Abril 16, 2023",Grêmio,1.65,1.73,2.95,1
...,...,...,...,...,...,...
367,"Sábado Mayo 27, 2023",Vasco da Gama,0.67,0.17,0.78,0
368,"Sábado Mayo 6, 2023",Cruzeiro,1.60,0.57,1.60,2
369,"Sábado Mayo 6, 2023",Fluminense,1.81,2.72,3.55,1
370,"Sábado Mayo 6, 2023",Santos,1.01,0.86,1.08,1


In [15]:
promedio_por_tiro_brasil=datos_brasil_shots.groupby(by=['Fecha','Equipo']).mean().reset_index()[['Fecha','Equipo','xG','PSxG','xGReal','Goles','Distance']]
promedio_por_tiro_brasil

,Fecha,Equipo,xG,PSxG,xGReal,Goles,Distance
0,"Domingo Abril 16, 2023",Corinthians,0.129167,0.321429,0.201667,0.166667,20.750000
1,"Domingo Abril 16, 2023",Coritiba,0.037000,0.140000,0.061000,0.000000,23.300000
2,"Domingo Abril 16, 2023",Cruzeiro,0.068750,0.270000,0.101250,0.125000,20.625000
3,"Domingo Abril 16, 2023",Flamengo,0.092000,0.371429,0.191333,0.200000,21.066667
4,"Domingo Abril 16, 2023",Grêmio,0.097059,0.288333,0.173529,0.058824,21.882353
...,...,...,...,...,...,...,...
367,"Sábado Mayo 27, 2023",Vasco da Gama,0.055833,0.170000,0.065000,0.000000,21.583333
368,"Sábado Mayo 6, 2023",Cruzeiro,0.160000,0.190000,0.160000,0.200000,18.700000
369,"Sábado Mayo 6, 2023",Fluminense,0.078696,0.302222,0.154348,0.043478,20.956522
370,"Sábado Mayo 6, 2023",Santos,0.112222,0.215000,0.120000,0.111111,18.222222


In [16]:
promedio_por_tiro_brasil.rename(columns={"xG":"xG_promedio_por_tiro","PSxG":"PSxG_promedio_por_tiro","xGReal":"xGReal_promedio_por_tiro","Goles":"Goles_promedio_por_tiro","Distance":"Distance_promedio_por_tiro"},inplace=True)

In [17]:
promedio_por_tiro_brasil

,Fecha,Equipo,xG_promedio_por_tiro,PSxG_promedio_por_tiro,xGReal_promedio_por_tiro,Goles_promedio_por_tiro,Distance_promedio_por_tiro
0,"Domingo Abril 16, 2023",Corinthians,0.129167,0.321429,0.201667,0.166667,20.750000
1,"Domingo Abril 16, 2023",Coritiba,0.037000,0.140000,0.061000,0.000000,23.300000
2,"Domingo Abril 16, 2023",Cruzeiro,0.068750,0.270000,0.101250,0.125000,20.625000
3,"Domingo Abril 16, 2023",Flamengo,0.092000,0.371429,0.191333,0.200000,21.066667
4,"Domingo Abril 16, 2023",Grêmio,0.097059,0.288333,0.173529,0.058824,21.882353
...,...,...,...,...,...,...,...
367,"Sábado Mayo 27, 2023",Vasco da Gama,0.055833,0.170000,0.065000,0.000000,21.583333
368,"Sábado Mayo 6, 2023",Cruzeiro,0.160000,0.190000,0.160000,0.200000,18.700000
369,"Sábado Mayo 6, 2023",Fluminense,0.078696,0.302222,0.154348,0.043478,20.956522
370,"Sábado Mayo 6, 2023",Santos,0.112222,0.215000,0.120000,0.111111,18.222222


In [18]:
tiros_reformados_brasil_f=tiros_reformados_brasil.merge(datos_brasil_shots[['Fecha','Equipo','LocalOVisita']].drop_duplicates(subset=['Fecha','Equipo']),on=['Fecha','Equipo'],how="left")
tiros_reformados_brasil_f

,Fecha,Equipo,xG,PSxG,xGReal,Goles,LocalOVisita
0,"Domingo Abril 16, 2023",Corinthians,1.55,2.25,2.42,2,Local
1,"Domingo Abril 16, 2023",Coritiba,0.37,0.42,0.61,0,Visitante
2,"Domingo Abril 16, 2023",Cruzeiro,0.55,0.54,0.81,1,Visitante
3,"Domingo Abril 16, 2023",Flamengo,1.38,2.60,2.87,3,Local
4,"Domingo Abril 16, 2023",Grêmio,1.65,1.73,2.95,1,Local
...,...,...,...,...,...,...,...
367,"Sábado Mayo 27, 2023",Vasco da Gama,0.67,0.17,0.78,0,Visitante
368,"Sábado Mayo 6, 2023",Cruzeiro,1.60,0.57,1.60,2,Local
369,"Sábado Mayo 6, 2023",Fluminense,1.81,2.72,3.55,1,Local
370,"Sábado Mayo 6, 2023",Santos,1.01,0.86,1.08,1,Visitante


In [19]:
tiros_reformados_brasil_f_f= tiros_reformados_brasil_f.merge(promedio_por_tiro_brasil,how="left",on=['Fecha','Equipo'])
tiros_reformados_brasil_f_f

,Fecha,Equipo,xG,PSxG,xGReal,Goles,LocalOVisita,xG_promedio_por_tiro,PSxG_promedio_por_tiro,xGReal_promedio_por_tiro,Goles_promedio_por_tiro,Distance_promedio_por_tiro
0,"Domingo Abril 16, 2023",Corinthians,1.55,2.25,2.42,2,Local,0.129167,0.321429,0.201667,0.166667,20.750000
1,"Domingo Abril 16, 2023",Coritiba,0.37,0.42,0.61,0,Visitante,0.037000,0.140000,0.061000,0.000000,23.300000
2,"Domingo Abril 16, 2023",Cruzeiro,0.55,0.54,0.81,1,Visitante,0.068750,0.270000,0.101250,0.125000,20.625000
3,"Domingo Abril 16, 2023",Flamengo,1.38,2.60,2.87,3,Local,0.092000,0.371429,0.191333,0.200000,21.066667
4,"Domingo Abril 16, 2023",Grêmio,1.65,1.73,2.95,1,Local,0.097059,0.288333,0.173529,0.058824,21.882353
...,...,...,...,...,...,...,...,...,...,...,...,...
367,"Sábado Mayo 27, 2023",Vasco da Gama,0.67,0.17,0.78,0,Visitante,0.055833,0.170000,0.065000,0.000000,21.583333
368,"Sábado Mayo 6, 2023",Cruzeiro,1.60,0.57,1.60,2,Local,0.160000,0.190000,0.160000,0.200000,18.700000
369,"Sábado Mayo 6, 2023",Fluminense,1.81,2.72,3.55,1,Local,0.078696,0.302222,0.154348,0.043478,20.956522
370,"Sábado Mayo 6, 2023",Santos,1.01,0.86,1.08,1,Visitante,0.112222,0.215000,0.120000,0.111111,18.222222


In [20]:
tiros_reformados_brasil_f_f.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 372 entries, 0 to 371
Data columns (total 12 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Fecha                       372 non-null    object 
 1   Equipo                      372 non-null    object 
 2   xG                          372 non-null    float64
 3   PSxG                        372 non-null    float64
 4   xGReal                      372 non-null    float64
 5   Goles                       372 non-null    int32  
 6   LocalOVisita                372 non-null    object 
 7   xG_promedio_por_tiro        372 non-null    float64
 8   PSxG_promedio_por_tiro      366 non-null    float64
 9   xGReal_promedio_por_tiro    372 non-null    float64
 10  Goles_promedio_por_tiro     372 non-null    float64
 11  Distance_promedio_por_tiro  372 non-null    float64
dtypes: float64(8), int32(1), object(3)
memory usage: 36.3+ KB


In [21]:
tiros_reformados_brasil_f_f.to_excel("Tiros_brasil_acum_fecha_equipo.xlsx")

# Agregar datos de tiros a partidos

In [22]:
datos_brasil_matchs.drop(inplace=True,columns=['Unnamed: 0'])
datos_brasil_matchs

,Fecha,HomeTeam,AwayTeam,HF,AF,HC,AC,HPCru,APCru,HTo,...,HOFF,AOFF,HGK,AGK,HSB,ASB,HPelotazos,APelotazos,HxG,AxG
0,"Sábado Abril 15, 2023",América (MG),Fluminense,25,9,11,1,43,6,506,...,2,3,4,16,15,13,68,57,"1,1","3,1"
1,"Sábado Abril 15, 2023",Palmeiras,Cuiabá,23,12,4,4,22,15,610,...,3,2,12,8,25,19,83,51,"3,0","1,0"
2,"Sábado Abril 15, 2023",Bragantino,Bahia,20,13,4,7,17,14,609,...,2,3,7,13,26,18,63,77,"2,0","0,7"
3,"Sábado Abril 15, 2023",Botafogo (RJ),São Paulo,20,13,3,7,15,18,444,...,2,1,7,6,19,10,55,71,"0,9","2,0"
4,"Sábado Abril 15, 2023",Atl Paranaense,Goiás,19,18,3,8,20,23,529,...,1,1,10,5,18,24,75,61,"0,8","1,0"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183,"Domingo Agosto 13, 2023",Flamengo,São Paulo,13,15,8,2,25,7,709,...,3,4,3,13,21,13,54,77,"1,4","0,8"
184,"Sábado Agosto 12, 2023",Botafogo (RJ),Internacional,15,10,7,3,24,14,647,...,0,4,5,6,19,15,77,55,"2,3","1,7"
185,"Lunes Agosto 14, 2023",Palmeiras,Cruzeiro,13,26,3,5,25,17,583,...,1,1,6,11,23,18,90,58,"1,3","0,7"
186,"Martes Agosto 15, 2023",Ath Paranaense,Cuiabá,22,12,5,3,15,16,556,...,1,1,12,9,18,10,88,59,"0,8","1,1"


In [23]:
tiros_reformados_brasil_f_f[(tiros_reformados_brasil_f_f['Equipo']=="Palmeiras")]

,Fecha,Equipo,xG,PSxG,xGReal,Goles,LocalOVisita,xG_promedio_por_tiro,PSxG_promedio_por_tiro,xGReal_promedio_por_tiro,Goles_promedio_por_tiro,Distance_promedio_por_tiro
13,"Domingo Abril 23, 2023",Palmeiras,0.67,1.57,1.82,2,Visitante,0.051538,0.224286,0.140000,0.153846,21.538462
55,"Domingo Julio 16, 2023",Palmeiras,1.47,1.84,2.53,0,Visitante,0.091875,0.368000,0.158125,0.000000,19.750000
65,"Domingo Julio 2, 2023",Palmeiras,1.63,2.34,2.65,2,Visitante,0.148182,0.468000,0.240909,0.181818,19.636364
80,"Domingo Julio 30, 2023",Palmeiras,0.92,1.71,1.95,4,Visitante,0.065714,0.244286,0.139286,0.285714,20.285714
99,"Domingo Junio 11, 2023",Palmeiras,0.68,1.21,1.54,2,Visitante,0.068000,0.302500,0.154000,0.200000,19.500000
110,"Domingo Junio 25, 2023",Palmeiras,2.58,0.38,2.18,0,Local,0.172000,0.095000,0.145333,0.000000,16.133333
118,"Domingo Junio 4, 2023",Palmeiras,2.49,3.81,4.75,3,Local,0.108261,0.476250,0.206522,0.130435,16.260870
144,"Domingo Mayo 28, 2023",Palmeiras,1.12,1.17,1.66,1,Visitante,0.101818,0.585000,0.150909,0.090909,18.909091
158,"Domingo Mayo 7, 2023",Palmeiras,1.98,2.96,3.55,4,Visitante,0.116471,0.422857,0.208824,0.235294,15.705882
180,"Lunes Agosto 14, 2023",Palmeiras,1.27,1.05,1.59,1,Local,0.105833,0.262500,0.132500,0.083333,18.666667


In [24]:
datos_brasil_matchs[(datos_brasil_matchs['Fecha']=="Domingo Abril 16, 2023") & (datos_brasil_matchs['AwayTeam']=="Santos")]

,Fecha,HomeTeam,AwayTeam,HF,AF,HC,AC,HPCru,APCru,HTo,...,HOFF,AOFF,HGK,AGK,HSB,ASB,HPelotazos,APelotazos,HxG,AxG
8,"Domingo Abril 16, 2023",Grêmio,Santos,12,16,4,11,11,31,576,...,1,1,13,7,16,25,60,62,"1,6","1,2"


In [25]:
datos_brasil_matchs[(datos_brasil_matchs['Fecha']=="Domingo Abril 16, 2023")]

,Fecha,HomeTeam,AwayTeam,HF,AF,HC,AC,HPCru,APCru,HTo,...,HOFF,AOFF,HGK,AGK,HSB,ASB,HPelotazos,APelotazos,HxG,AxG
6,"Domingo Abril 16, 2023",Flamengo,Coritiba,20,17,6,3,16,12,704,...,1,2,3,5,19,21,58,58,"1,4","0,4"
7,"Domingo Abril 16, 2023",Corinthians,Cruzeiro,14,12,7,4,15,24,510,...,4,3,5,8,16,31,64,60,"1,5","0,5"
8,"Domingo Abril 16, 2023",Grêmio,Santos,12,16,4,11,11,31,576,...,1,1,13,7,16,25,60,62,"1,6","1,2"


In [26]:
infoLocales=[]
infoVisitas=[]
for fecha,equiLocal,equiVisita in zip(datos_brasil_matchs['Fecha'],datos_brasil_matchs['HomeTeam'],datos_brasil_matchs['AwayTeam']):
    info_local_tiros=tiros_reformados_brasil_f_f[(tiros_reformados_brasil_f_f['Fecha']==fecha) & (tiros_reformados_brasil_f_f['Equipo']==equiLocal)]
    info_visita_tiros=tiros_reformados_brasil_f_f[(tiros_reformados_brasil_f_f['Fecha']==fecha) & (tiros_reformados_brasil_f_f['Equipo']==equiVisita)]
    if len(infoLocales)==0:
        infoLocales= info_local_tiros.copy()
    else:
        infoLocales = pd.concat([infoLocales,info_local_tiros])
        
    if len(infoVisitas)==0:
        infoVisitas= info_visita_tiros.copy()
    else:
        infoVisitas = pd.concat([infoVisitas,info_visita_tiros])
    

In [27]:
print(len(infoLocales))
print(len(infoVisitas))

188
188


In [28]:
datos_final_partidos_fbref_brasil=(datos_brasil_matchs.merge(infoLocales,how="left",right_on=['Fecha','Equipo'],left_on=['Fecha','HomeTeam'])).merge(infoVisitas,how="left",right_on=['Fecha','Equipo'],left_on=['Fecha','AwayTeam'],suffixes=('_Local', '_Visitante'))

In [29]:
datos_final_partidos_fbref_brasil

,Fecha,HomeTeam,AwayTeam,HF,AF,HC,AC,HPCru,APCru,HTo,...,xG_Visitante,PSxG_Visitante,xGReal_Visitante,Goles_Visitante,LocalOVisita_Visitante,xG_promedio_por_tiro_Visitante,PSxG_promedio_por_tiro_Visitante,xGReal_promedio_por_tiro_Visitante,Goles_promedio_por_tiro_Visitante,Distance_promedio_por_tiro_Visitante
0,"Sábado Abril 15, 2023",América (MG),Fluminense,25,9,11,1,43,6,506,...,3.12,4.23,4.56,3,Visitante,0.208000,0.52875,0.304000,0.200000,17.533333
1,"Sábado Abril 15, 2023",América (MG),Fluminense,25,9,11,1,43,6,506,...,3.12,4.23,4.56,3,Visitante,0.208000,0.52875,0.304000,0.200000,17.533333
2,"Sábado Abril 15, 2023",América (MG),Fluminense,25,9,11,1,43,6,506,...,3.12,4.23,4.56,3,Visitante,0.208000,0.52875,0.304000,0.200000,17.533333
3,"Sábado Abril 15, 2023",América (MG),Fluminense,25,9,11,1,43,6,506,...,3.12,4.23,4.56,3,Visitante,0.208000,0.52875,0.304000,0.200000,17.533333
4,"Sábado Abril 15, 2023",Palmeiras,Cuiabá,23,12,4,4,22,15,610,...,0.99,1.56,2.19,1,Visitante,0.082500,0.39000,0.182500,0.083333,17.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,"Domingo Agosto 13, 2023",Flamengo,São Paulo,13,15,8,2,25,7,709,...,0.77,0.23,0.96,1,Visitante,0.096250,0.23000,0.120000,0.125000,21.000000
196,"Sábado Agosto 12, 2023",Botafogo (RJ),Internacional,15,10,7,3,24,14,647,...,1.73,1.73,2.06,1,Visitante,0.173000,0.43250,0.206000,0.100000,15.700000
197,"Lunes Agosto 14, 2023",Palmeiras,Cruzeiro,13,26,3,5,25,17,583,...,0.66,0.57,1.17,0,Visitante,0.050769,0.11400,0.090000,0.000000,24.461538
198,"Martes Agosto 15, 2023",Ath Paranaense,Cuiabá,22,12,5,3,15,16,556,...,1.10,0.21,1.28,0,Visitante,0.100000,0.21000,0.116364,0.000000,24.454545


In [30]:
datos_final_partidos_fbref_brasil[datos_final_partidos_fbref_brasil['HomeTeam']=="Palmeiras"]

,Fecha,HomeTeam,AwayTeam,HF,AF,HC,AC,HPCru,APCru,HTo,...,xG_Visitante,PSxG_Visitante,xGReal_Visitante,Goles_Visitante,LocalOVisita_Visitante,xG_promedio_por_tiro_Visitante,PSxG_promedio_por_tiro_Visitante,xGReal_promedio_por_tiro_Visitante,Goles_promedio_por_tiro_Visitante,Distance_promedio_por_tiro_Visitante
4,"Sábado Abril 15, 2023",Palmeiras,Cuiabá,23,12,4,4,22,15,610,...,0.99,1.56,2.19,1,Visitante,0.082500,0.3900,0.182500,0.083333,17.666667
25,"Sábado Abril 29, 2023",Palmeiras,Corinthians,20,15,12,3,21,13,663,...,0.51,0.14,0.56,0,Visitante,0.051000,0.0700,0.056000,0.000000,23.600000
46,"Miércoles Mayo 10, 2023",Palmeiras,Grêmio,10,13,7,4,24,8,709,...,0.07,0.50,0.52,1,Visitante,0.023333,0.2500,0.173333,0.333333,22.000000
50,"Sábado Mayo 13, 2023",Palmeiras,Bragantino,12,19,15,6,33,15,589,...,0.97,0.27,1.13,1,Visitante,0.121250,0.0900,0.141250,0.125000,21.625000
82,"Domingo Junio 4, 2023",Palmeiras,Coritiba,15,13,11,6,38,19,576,...,0.59,0.79,0.94,1,Visitante,0.084286,0.1975,0.134286,0.142857,21.428571
106,"Domingo Junio 25, 2023",Palmeiras,Botafogo (RJ),17,14,12,1,48,8,640,...,0.78,0.47,1.14,1,Visitante,0.130000,0.2350,0.190000,0.166667,18.166667
128,"Sábado Julio 8, 2023",Palmeiras,Flamengo,13,16,4,7,21,21,523,...,0.84,1.42,1.75,1,Visitante,0.084000,0.3550,0.175000,0.100000,17.000000
137,"Sábado Julio 22, 2023",Palmeiras,Fortaleza,17,28,4,3,20,9,569,...,1.53,2.40,2.51,1,Visitante,0.139091,0.4000,0.228182,0.090909,24.454545
197,"Lunes Agosto 14, 2023",Palmeiras,Cruzeiro,13,26,3,5,25,17,583,...,0.66,0.57,1.17,0,Visitante,0.050769,0.1140,0.090000,0.000000,24.461538


In [31]:
datos_final_partidos_fbref_brasil.iloc[0]

Fecha                                   Sábado Abril 15, 2023
HomeTeam                                         América (MG)
AwayTeam                                           Fluminense
HF                                                         25
AF                                                          9
HC                                                         11
AC                                                          1
HPCru                                                      43
APCru                                                       6
HTo                                                       506
ATo                                                       583
HDerribo                                                   19
ADerribo                                                   15
HInter                                                     16
AInter                                                      9
HDAG                                                        8
ADAG    

# Añadir datos de mismarcadores con la información por partido de Fbref

In [32]:
datos_mis_marcadores_brasil= pd.read_excel("./data/brasil_brasileirao-serie-a/version0/resultados_anteriores.xlsx")
datos_mis_marcadores_brasil

,Date,HomeTeam,AwayTeam,HG,AG,HxG,AxG,HP,AP,HTS,...,ATP,HPC,APC,HT,AT,HA,AA,HDA,ADA,Resultado
0,15.08.2023 18:00,Athletico-PR,Cuiabá,2,0,0.82,1.12,55%,45%,12,...,329.0,362.0,269.0,NaN,NaN,93,105,41,34,2
1,14.08.2023 19:00,Bragantino,Vasco,1,1,1.49,1.39,60%,40%,22,...,267.0,413.0,187.0,NaN,NaN,148,76,75,31,0
2,14.08.2023 17:00,Palmeiras,Cruzeiro,1,0,1.15,0.63,54%,46%,11,...,395.0,376.0,319.0,NaN,NaN,105,115,40,27,1
3,13.08.2023 16:30,Flamengo,Sao Paulo,1,1,1.49,0.83,47%,53%,17,...,494.0,519.0,430.0,NaN,NaN,129,99,83,25,0
4,13.08.2023 16:30,Fortaleza,Santos,4,0,2.87,0.53,53%,47%,25,...,308.0,323.0,240.0,NaN,NaN,96,74,56,33,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183,15.04.2023 16:30,Botafogo,Sao Paulo,2,1,NaN,NaN,32%,68%,12,...,646.0,229.0,574.0,10.0,16.0,85,122,57,41,1
184,15.04.2023 16:30,Bragantino,Bahia,2,1,NaN,NaN,56%,44%,13,...,343.0,339.0,248.0,26.0,11.0,127,87,54,39,1
185,15.04.2023 16:30,Fortaleza,Internacional,1,1,NaN,NaN,47%,53%,24,...,474.0,319.0,379.0,17.0,19.0,126,110,73,30,0
186,15.04.2023 14:00,América-MG,Fluminense,0,3,1.14,3.26,46%,54%,17,...,442.0,267.0,364.0,18.0,14.0,114,56,95,22,-3


In [33]:
datos_final_partidos_fbref_brasil

,Fecha,HomeTeam,AwayTeam,HF,AF,HC,AC,HPCru,APCru,HTo,...,xG_Visitante,PSxG_Visitante,xGReal_Visitante,Goles_Visitante,LocalOVisita_Visitante,xG_promedio_por_tiro_Visitante,PSxG_promedio_por_tiro_Visitante,xGReal_promedio_por_tiro_Visitante,Goles_promedio_por_tiro_Visitante,Distance_promedio_por_tiro_Visitante
0,"Sábado Abril 15, 2023",América (MG),Fluminense,25,9,11,1,43,6,506,...,3.12,4.23,4.56,3,Visitante,0.208000,0.52875,0.304000,0.200000,17.533333
1,"Sábado Abril 15, 2023",América (MG),Fluminense,25,9,11,1,43,6,506,...,3.12,4.23,4.56,3,Visitante,0.208000,0.52875,0.304000,0.200000,17.533333
2,"Sábado Abril 15, 2023",América (MG),Fluminense,25,9,11,1,43,6,506,...,3.12,4.23,4.56,3,Visitante,0.208000,0.52875,0.304000,0.200000,17.533333
3,"Sábado Abril 15, 2023",América (MG),Fluminense,25,9,11,1,43,6,506,...,3.12,4.23,4.56,3,Visitante,0.208000,0.52875,0.304000,0.200000,17.533333
4,"Sábado Abril 15, 2023",Palmeiras,Cuiabá,23,12,4,4,22,15,610,...,0.99,1.56,2.19,1,Visitante,0.082500,0.39000,0.182500,0.083333,17.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,"Domingo Agosto 13, 2023",Flamengo,São Paulo,13,15,8,2,25,7,709,...,0.77,0.23,0.96,1,Visitante,0.096250,0.23000,0.120000,0.125000,21.000000
196,"Sábado Agosto 12, 2023",Botafogo (RJ),Internacional,15,10,7,3,24,14,647,...,1.73,1.73,2.06,1,Visitante,0.173000,0.43250,0.206000,0.100000,15.700000
197,"Lunes Agosto 14, 2023",Palmeiras,Cruzeiro,13,26,3,5,25,17,583,...,0.66,0.57,1.17,0,Visitante,0.050769,0.11400,0.090000,0.000000,24.461538
198,"Martes Agosto 15, 2023",Ath Paranaense,Cuiabá,22,12,5,3,15,16,556,...,1.10,0.21,1.28,0,Visitante,0.100000,0.21000,0.116364,0.000000,24.454545


In [34]:
datos_mis_marcadores_brasil.sort_values(by="HomeTeam",ascending=True)['HomeTeam'].unique()

array(['América-MG', 'Athletico-PR', 'Atlético-MG', 'Bahia', 'Botafogo',
       'Bragantino', 'Corinthians', 'Coritiba', 'Cruzeiro', 'Cuiabá',
       'Flamengo', 'Fluminense', 'Fortaleza', 'Goiás', 'Gremio',
       'Internacional', 'Palmeiras', 'Santos', 'Sao Paulo', 'Vasco'],
      dtype=object)

In [35]:
datos_final_partidos_fbref_brasil.sort_values(by="HomeTeam",ascending=True)['HomeTeam'].unique()

array(['América (MG)', 'Ath Paranaense', 'Atl Paranaense',
       'Atlético Mineiro', 'Bahia', 'Botafogo (RJ)', 'Bragantino',
       'Corinthians', 'Coritiba', 'Cruzeiro', 'Cuiabá', 'Flamengo',
       'Fluminense', 'Fortaleza', 'Goiás', 'Grêmio', 'Internacional',
       'Palmeiras', 'Santos', 'São Paulo', 'Vasco da Gama'], dtype=object)

In [36]:
datos_final_partidos_fbref_brasil_f=datos_final_partidos_fbref_brasil.drop(columns=["HF","AF","HC","AC","HOFF","AOFF","HSB","ASB"])

In [37]:
datos_final_partidos_fbref_brasil_f

,Fecha,HomeTeam,AwayTeam,HPCru,APCru,HTo,ATo,HDerribo,ADerribo,HInter,...,xG_Visitante,PSxG_Visitante,xGReal_Visitante,Goles_Visitante,LocalOVisita_Visitante,xG_promedio_por_tiro_Visitante,PSxG_promedio_por_tiro_Visitante,xGReal_promedio_por_tiro_Visitante,Goles_promedio_por_tiro_Visitante,Distance_promedio_por_tiro_Visitante
0,"Sábado Abril 15, 2023",América (MG),Fluminense,43,6,506,583,19,15,16,...,3.12,4.23,4.56,3,Visitante,0.208000,0.52875,0.304000,0.200000,17.533333
1,"Sábado Abril 15, 2023",América (MG),Fluminense,43,6,506,583,19,15,16,...,3.12,4.23,4.56,3,Visitante,0.208000,0.52875,0.304000,0.200000,17.533333
2,"Sábado Abril 15, 2023",América (MG),Fluminense,43,6,506,583,19,15,16,...,3.12,4.23,4.56,3,Visitante,0.208000,0.52875,0.304000,0.200000,17.533333
3,"Sábado Abril 15, 2023",América (MG),Fluminense,43,6,506,583,19,15,16,...,3.12,4.23,4.56,3,Visitante,0.208000,0.52875,0.304000,0.200000,17.533333
4,"Sábado Abril 15, 2023",Palmeiras,Cuiabá,22,15,610,578,19,22,8,...,0.99,1.56,2.19,1,Visitante,0.082500,0.39000,0.182500,0.083333,17.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,"Domingo Agosto 13, 2023",Flamengo,São Paulo,25,7,709,631,19,13,12,...,0.77,0.23,0.96,1,Visitante,0.096250,0.23000,0.120000,0.125000,21.000000
196,"Sábado Agosto 12, 2023",Botafogo (RJ),Internacional,24,14,647,624,15,15,15,...,1.73,1.73,2.06,1,Visitante,0.173000,0.43250,0.206000,0.100000,15.700000
197,"Lunes Agosto 14, 2023",Palmeiras,Cruzeiro,25,17,583,550,17,12,10,...,0.66,0.57,1.17,0,Visitante,0.050769,0.11400,0.090000,0.000000,24.461538
198,"Martes Agosto 15, 2023",Ath Paranaense,Cuiabá,15,16,556,497,16,13,5,...,1.10,0.21,1.28,0,Visitante,0.100000,0.21000,0.116364,0.000000,24.454545


In [38]:
def normalizar_nombres_equipos(entrada):
    if entrada=='América (MG)':
        return "América-MG"
    elif entrada=='Atl Paranaense':
        return "Athletico-PR"
    elif entrada=='Atlético Mineiro':
        return "Atlético-MG"
    elif entrada=="Botafogo (RJ)":
        return "Botafogo"
    elif entrada=="Grêmio":
        return "Gremio"
    elif entrada=="São Paulo":
        return "Sao Paulo"
    elif entrada=="Vasco da Gama":
        return "Vasco"
    else:
        return entrada

In [39]:
def quitar_espacios(entrada):
    return entrada.strip()

In [40]:
def punto_por_coma(entrada):
    return entrada.replace(",",".")

In [41]:
datos_final_partidos_fbref_brasil_f['HxG'] = datos_final_partidos_fbref_brasil_f['HxG'].apply(punto_por_coma).astype(float)
datos_final_partidos_fbref_brasil_f['AxG'] = datos_final_partidos_fbref_brasil_f['AxG'].apply(punto_por_coma).astype(float)

In [42]:
# info_fbref_match_stats[(info_fbref_match_stats['HomeTeam']=='Newcastle') & (info_fbref_match_stats['AwayTeam']=="Nott'ham Forest")]

In [43]:
datos_final_partidos_fbref_brasil_f['HomeTeam'] = datos_final_partidos_fbref_brasil_f['HomeTeam'].apply(quitar_espacios)
datos_final_partidos_fbref_brasil_f['AwayTeam'] = datos_final_partidos_fbref_brasil_f['AwayTeam'].apply(quitar_espacios)
datos_mis_marcadores_brasil['HomeTeam'] = datos_mis_marcadores_brasil['HomeTeam'].apply(quitar_espacios)
datos_mis_marcadores_brasil['AwayTeam'] = datos_mis_marcadores_brasil['AwayTeam'].apply(quitar_espacios)
datos_final_partidos_fbref_brasil_f['HomeTeam'] = datos_final_partidos_fbref_brasil_f['HomeTeam'].apply(normalizar_nombres_equipos)
datos_final_partidos_fbref_brasil_f['AwayTeam'] = datos_final_partidos_fbref_brasil_f['AwayTeam'].apply(normalizar_nombres_equipos)

In [44]:
datos_mis_marcadores_brasil.sort_values(by="HomeTeam",ascending=True)['HomeTeam'].unique()

array(['América-MG', 'Athletico-PR', 'Atlético-MG', 'Bahia', 'Botafogo',
       'Bragantino', 'Corinthians', 'Coritiba', 'Cruzeiro', 'Cuiabá',
       'Flamengo', 'Fluminense', 'Fortaleza', 'Goiás', 'Gremio',
       'Internacional', 'Palmeiras', 'Santos', 'Sao Paulo', 'Vasco'],
      dtype=object)

In [45]:
datos_final_partidos_fbref_brasil_f.sort_values(by="HomeTeam",ascending=True)['HomeTeam'].unique()

array(['América-MG', 'Ath Paranaense', 'Athletico-PR', 'Atlético-MG',
       'Bahia', 'Botafogo', 'Bragantino', 'Corinthians', 'Coritiba',
       'Cruzeiro', 'Cuiabá', 'Flamengo', 'Fluminense', 'Fortaleza',
       'Goiás', 'Gremio', 'Internacional', 'Palmeiras', 'Santos',
       'Sao Paulo', 'Vasco'], dtype=object)

In [46]:
datos_final_partidos_fbref_brasil_f.drop(columns=['LocalOVisita_Visitante','LocalOVisita_Local'],inplace=True)

In [47]:
datos_final_partidos_fbref_brasil_f[datos_final_partidos_fbref_brasil_f['HomeTeam']=="Palmeiras"]

,Fecha,HomeTeam,AwayTeam,HPCru,APCru,HTo,ATo,HDerribo,ADerribo,HInter,...,Equipo_Visitante,xG_Visitante,PSxG_Visitante,xGReal_Visitante,Goles_Visitante,xG_promedio_por_tiro_Visitante,PSxG_promedio_por_tiro_Visitante,xGReal_promedio_por_tiro_Visitante,Goles_promedio_por_tiro_Visitante,Distance_promedio_por_tiro_Visitante
4,"Sábado Abril 15, 2023",Palmeiras,Cuiabá,22,15,610,578,19,22,8,...,Cuiabá,0.99,1.56,2.19,1,0.082500,0.3900,0.182500,0.083333,17.666667
25,"Sábado Abril 29, 2023",Palmeiras,Corinthians,21,13,663,571,19,12,10,...,Corinthians,0.51,0.14,0.56,0,0.051000,0.0700,0.056000,0.000000,23.600000
46,"Miércoles Mayo 10, 2023",Palmeiras,Gremio,24,8,709,515,7,10,10,...,Grêmio,0.07,0.50,0.52,1,0.023333,0.2500,0.173333,0.333333,22.000000
50,"Sábado Mayo 13, 2023",Palmeiras,Bragantino,33,15,589,488,19,12,11,...,Bragantino,0.97,0.27,1.13,1,0.121250,0.0900,0.141250,0.125000,21.625000
82,"Domingo Junio 4, 2023",Palmeiras,Coritiba,38,19,576,568,12,14,11,...,Coritiba,0.59,0.79,0.94,1,0.084286,0.1975,0.134286,0.142857,21.428571
106,"Domingo Junio 25, 2023",Palmeiras,Botafogo,48,8,640,444,17,19,5,...,Botafogo (RJ),0.78,0.47,1.14,1,0.130000,0.2350,0.190000,0.166667,18.166667
128,"Sábado Julio 8, 2023",Palmeiras,Flamengo,21,21,523,658,18,8,4,...,Flamengo,0.84,1.42,1.75,1,0.084000,0.3550,0.175000,0.100000,17.000000
137,"Sábado Julio 22, 2023",Palmeiras,Fortaleza,20,9,569,345,13,10,10,...,Fortaleza,1.53,2.40,2.51,1,0.139091,0.4000,0.228182,0.090909,24.454545
197,"Lunes Agosto 14, 2023",Palmeiras,Cruzeiro,25,17,583,550,17,12,10,...,Cruzeiro,0.66,0.57,1.17,0,0.050769,0.1140,0.090000,0.000000,24.461538


In [48]:
datos_final_partidos_fbref_brasil_f=datos_final_partidos_fbref_brasil_f.drop_duplicates(subset=['Fecha','HomeTeam','AwayTeam']).copy()

In [49]:
info_final= datos_mis_marcadores_brasil.merge(datos_final_partidos_fbref_brasil_f,how="left",on=['HomeTeam',"AwayTeam"])
info_final

,Date,HomeTeam,AwayTeam,HG,AG,HxG_x,AxG_x,HP,AP,HTS,...,Equipo_Visitante,xG_Visitante,PSxG_Visitante,xGReal_Visitante,Goles_Visitante,xG_promedio_por_tiro_Visitante,PSxG_promedio_por_tiro_Visitante,xGReal_promedio_por_tiro_Visitante,Goles_promedio_por_tiro_Visitante,Distance_promedio_por_tiro_Visitante
0,15.08.2023 18:00,Athletico-PR,Cuiabá,2,0,0.82,1.12,55%,45%,12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,14.08.2023 19:00,Bragantino,Vasco,1,1,1.49,1.39,60%,40%,22,...,Vasco da Gama,1.54,1.59,2.06,1.0,0.192500,0.397500,0.257500,0.125000,15.375000
2,14.08.2023 17:00,Palmeiras,Cruzeiro,1,0,1.15,0.63,54%,46%,11,...,Cruzeiro,0.66,0.57,1.17,0.0,0.050769,0.114000,0.090000,0.000000,24.461538
3,13.08.2023 16:30,Flamengo,Sao Paulo,1,1,1.49,0.83,47%,53%,17,...,São Paulo,0.77,0.23,0.96,1.0,0.096250,0.230000,0.120000,0.125000,21.000000
4,13.08.2023 16:30,Fortaleza,Santos,4,0,2.87,0.53,53%,47%,25,...,Santos,0.52,0.94,1.37,0.0,0.057778,0.313333,0.152222,0.000000,17.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183,15.04.2023 16:30,Botafogo,Sao Paulo,2,1,NaN,NaN,32%,68%,12,...,São Paulo,2.10,1.39,2.20,1.0,0.131250,0.173750,0.137500,0.062500,16.437500
184,15.04.2023 16:30,Bragantino,Bahia,2,1,NaN,NaN,56%,44%,13,...,Bahia,0.65,0.95,1.21,1.0,0.072222,0.316667,0.134444,0.111111,19.222222
185,15.04.2023 16:30,Fortaleza,Internacional,1,1,NaN,NaN,47%,53%,24,...,Internacional,0.93,1.09,1.41,1.0,0.084545,0.181667,0.128182,0.090909,18.909091
186,15.04.2023 14:00,América-MG,Fluminense,0,3,1.14,3.26,46%,54%,17,...,Fluminense,3.12,4.23,4.56,3.0,0.208000,0.528750,0.304000,0.200000,17.533333


In [50]:
info_final[['HxG_x','AxG_x','HxG_y','AxG_y']]

,HxG_x,AxG_x,HxG_y,AxG_y
0,0.82,1.12,NaN,NaN
1,1.49,1.39,1.6,1.5
2,1.15,0.63,1.3,0.7
3,1.49,0.83,1.4,0.8
4,2.87,0.53,2.6,0.5
...,...,...,...,...
183,NaN,NaN,0.9,2.0
184,NaN,NaN,2.0,0.7
185,NaN,NaN,1.5,0.9
186,1.14,3.26,1.1,3.1


In [51]:
info_final.to_excel("info_partidos_completo_brasil.xlsx")

In [52]:
info_final_f=info_final.dropna(subset=['HxG_y'])
info_final_f.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 183 entries, 1 to 187
Data columns (total 83 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Date                                  183 non-null    object 
 1   HomeTeam                              183 non-null    object 
 2   AwayTeam                              183 non-null    object 
 3   HG                                    183 non-null    int64  
 4   AG                                    183 non-null    int64  
 5   HxG_x                                 174 non-null    float64
 6   AxG_x                                 174 non-null    float64
 7   HP                                    183 non-null    object 
 8   AP                                    183 non-null    object 
 9   HTS                                   183 non-null    int64  
 10  ATS                                   183 non-null    int64  
 11  HSI                

In [53]:
info_final_f=info_final_f.fillna(0)

# Creación Modelo OLS

In [54]:
info_final_f['Resultado']

1      0
2      1
3      0
4      4
5     -1
      ..
183    1
184    1
185    0
186   -3
187    1
Name: Resultado, Length: 183, dtype: int64

In [57]:
info_final_f['Resultado_conv'] = np.where(info_final_f['Resultado']>0,1,np.where(info_final_f['Resultado']<0,-1,0))

In [58]:
info_final_f['Resultado_conv']

1      0
2      1
3      0
4      1
5     -1
      ..
183    1
184    1
185    0
186   -1
187    1
Name: Resultado_conv, Length: 183, dtype: int32

In [59]:
string = ""
for i in info_final_f.drop(columns=['HxG_x','AxG_x','HGK','AGK','Date','HomeTeam','AwayTeam','Equipo_Local','Equipo_Visitante','Fecha','Resultado','Goles_Local','Goles_Visitante','HG','AG','HP','AP','HT','AT']).columns:
    string=string+ i +"+"
string= string[:-1]
string

'HTS+ATS+HSI+ASI+HSO+ASO+HBS+ABS+HFK+AFK+HC+AC+HOFF+AOFF+HTI+ATI+HGS+AGS+HF+AF+HRC+ARC+HYC+AYC+HTP+ATP+HPC+APC+HA+AA+HDA+ADA+HPCru+APCru+HTo+ATo+HDerribo+ADerribo+HInter+AInter+HDAG+ADAG+HDespeje+ADespeje+HPelotazos+APelotazos+HxG_y+AxG_y+xG_Local+PSxG_Local+xGReal_Local+xG_promedio_por_tiro_Local+PSxG_promedio_por_tiro_Local+xGReal_promedio_por_tiro_Local+Goles_promedio_por_tiro_Local+Distance_promedio_por_tiro_Local+xG_Visitante+PSxG_Visitante+xGReal_Visitante+xG_promedio_por_tiro_Visitante+PSxG_promedio_por_tiro_Visitante+xGReal_promedio_por_tiro_Visitante+Goles_promedio_por_tiro_Visitante+Distance_promedio_por_tiro_Visitante+Resultado_conv'

In [61]:
lm = smf.ols(formula = 'Resultado ~ '+string, data=info_final_f).fit()
lm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              Resultado   R-squared:                       0.996
Model:                            OLS   Adj. R-squared:                  0.994
Method:                 Least Squares   F-statistic:                     467.1
Date:                Mon, 04 Sep 2023   Prob (F-statistic):          2.63e-119
Time:                        19:13:15   Log-Likelihood:                 162.77
No. Observations:                 183   AIC:                            -197.5
Df Residuals:                     119   BIC:                             7.865
Df Model:                          63                                         
Covariance Type:            nonrobust                                         
========================================================================================================
                                           coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------
Intercept                                0.1901      0.325      0.584      0.560      -0.454       0.835
HTS                                      0.2454      0.010     24.879      0.000       0.226       0.265
ATS                                     -0.2269      0.010    -23.348      0.000      -0.246      -0.208
HSI                                      0.7257      0.028     26.359      0.000       0.671       0.780
ASI                                     -0.6908      0.025    -27.947      0.000      -0.740      -0.642
HSO                                     -0.2449      0.010    -23.451      0.000      -0.266      -0.224
ASO                                      0.2299      0.009     24.700      0.000       0.211       0.248
HBS                                     -0.2354      0.010    -23.876      0.000      -0.255      -0.216
ABS                                      0.2340      0.010     23.778      0.000       0.215       0.253
HFK                                      0.0074      0.007      0.994      0.322      -0.007       0.022
AFK                                     -0.0073      0.006     -1.162      0.248      -0.020       0.005
HC                                      -0.0018      0.006     -0.315      0.754      -0.013       0.009
AC                                      -0.0053      0.007     -0.798      0.426      -0.018       0.008
HOFF                                     0.0058      0.009      0.675      0.501      -0.011       0.023
AOFF                                     0.0092      0.010      0.934      0.352      -0.010       0.029
HTI                                      0.0038      0.002      1.541      0.126      -0.001       0.009
ATI                                     -0.0006      0.002     -0.245      0.807      -0.005       0.004
HGS                                      0.9330      0.031     29.848      0.000       0.871       0.995
AGS                                     -0.9810      0.036    -26.993      0.000      -1.053      -0.909
HF                                       0.0039      0.006      0.601      0.549      -0.009       0.017
AF                                      -0.0099      0.007     -1.366      0.174      -0.024       0.004
HRC                                      0.0235      0.037      0.631      0.529      -0.050       0.097
ARC                                      0.0114      0.038      0.301      0.764      -0.064       0.086
HYC                                     -0.0134      0.008     -1.639      0.104      -0.030       0.003
AYC                                      0.0132      0.008      1.650      0.102      -0.003       0.029
HTP                                      0.0017      0.001      1.365      0.175      -0.001       0.004
ATP                                   